# Определение стоимости автомобилей

## Подготовка данных

### Обзор

In [ ]:
!pip install scikit-learn==1.1.3

In [ ]:
!pip install phik

In [ ]:
import warnings

import pandas as pd
import phik
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from random import choices

from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV
)
from sklearn.metrics import (
    mean_squared_error
)

from sklearn.linear_model import(
    LinearRegression,
    LogisticRegression
)

from sklearn.preprocessing import (
    OneHotEncoder,
    MinMaxScaler
)  

from catboost import (
    CatBoostRegressor,
    Pool,
    cv
)

import lightgbm as lgb

from lightgbm import(
    LGBMRegressor
)

In [ ]:
# настройки
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

# константы заглавными буквами
RANDOM_STATE = 123456

In [ ]:
data = pd.read_csv('/datasets/autos.csv', parse_dates=['DateCrawled', 'DateCreated', 'LastSeen'])

In [ ]:
data.sample(5)

Для удобства привел к нижнему регистру названия столбцов

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
data.describe()

In [ ]:
data.boxplot(vert=False, figsize=(10, 12));

In [ ]:
data.hist(figsize=(15, 9));

Аномалии и ошибки в годе регистрации и в мощности двигателя.

### int64 в int16

In [ ]:
data[data.select_dtypes(['int']).columns] = data[data.select_dtypes(['int']).columns].astype('int16')

### Дубликаты

Удалил дубликаты строк

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

### Пропуски

In [ ]:
def nan_percent_plot(df):
    nan_percent = df.isna().mean()*100
    if nan_percent[nan_percent.values > 0].count() > 0:
        (
            nan_percent[nan_percent.values > 0]
            .sort_values(ascending=True)
            .plot(kind = 'barh', figsize=(16, 6), fontsize=22).set_title('Процент пропущенных значений' + "\n", fontsize=22)
        );
    else:
        print('Пропусков не найдено')

In [ ]:
nan_percent_plot(data)

Так как столбец `Repaired` критически важен для ценообразования с точки зрения человека, терпеть пропуски в данном столбце нельзя, даже несмотря на большое количество пропусков в датасете их следует удалить. Хотя даже для анализа можно было бы заменить пропуски на наиболее часто встречающуюся категорию, или вычислить вероятность категории из имеющихся данных и подставить в каждую строку определенное значение со своей вероятностью в зависимости от дргуих признаков, такой способ, например, сохранит стандартное отклонение если признак был бы числовой. Но для конкретной задачи регрессии такое большое количество синтетических данных для обучения реальной модели - это выстрел себе в ногу. Тут остается провести доп работу по исследовательскому анализу данного столбца в контексте данного датасета для наиболее правильного заполнения пропусков синтетикой - очень трудоемкий процесс, словно отдельный проект, либо просто дропнуть эти строи и не усложнять себе жизнь. Поэтому, я принял волевое решение - удалить эти строки из датасета, оставшихся вполне себе достаточно для обучения.

In [ ]:
data = data[~data['Repaired'].isna()]

Также привел к целочисленному типу данный признак.

In [ ]:
data['Repaired'] = data['Repaired'].apply(lambda ser: 1 if ser == 'yes' else 0).astype('int16')

Считаю пропуски в наименовании модели для задачи определения цены недопустимыми, удалил их из датасета, около пяти процентов от датасета.

In [ ]:
data = data[~data['Model'].isna()]

Пропуски в типе кузова можно считать не критичными, но есть сомнения, так как признак категориальный и если заполнить пропуски словом ""undefined" то машина будет считать это отдельной категорией. ~~Поэтому просто удалю все.~~ Как выяснилось далее, тип кузова очень сильно зависит от модели. Поэтому можно рассчитать вероятности типов кузовов для каждой модели и случайным выбором из списка, с помощью random choices, с рассчитанной вероятностью для каждого типа кузова заполнить пропуски.

In [ ]:
for model in data['Model'].unique():
    
    counts = data[(~data['VehicleType'].isna() & (data['Model'] == model))]['VehicleType'].value_counts()
    names = counts.index
    counts = counts.values
    
    proba = []
    for count in counts:
        proba.append(round(count / sum(counts), 3))
    
    # Индекс [0] в choices нужно указать, чтобы он вернул значение без квадратных скобочек
    data.loc[(data['VehicleType'].isna() & (data['Model'] == model)), 'VehicleType'] = \
    data.loc[(data['VehicleType'].isna() & (data['Model'] == model)), 'VehicleType'].apply(lambda ser: choices(names, weights=proba)[0])

Про оставшиеся два признака (тип топлива и коробка передач) частично можно сказать тоже что и выше, только не заполнить в них пропуски опираясь на другой столбец, значимой связи нет, поэтому удалю все оставшиеся пропуски. В дальнейшем была заметна некоторая связь между коробкой передач и моделью, поэтому операцию выше можно попробовать повторить и для коробки передач, но это уже эксперименты.

In [ ]:
data = data.dropna()

На этапе пропусков было удалено около ста тысяч записей, это около 29 процентов от всего датасета, но записей еще много, так что могу себе позволить удалить некачественные строки. При продакшине пользователь должен указать все данные, чтобы получить ответ от модели на определение рыночной стоимости. Можно конечно, оставить отдельную категорию "undefined" для некоторых признаков, чтобы пользователю не обязательно было вводить все данные. Но если модель планируется более менее серьезьной для серьезной работы то и пользователи должны отнестись серьезнее к заполнению данных, чтобы получить наиболее точный ответ от разработанной модели.

### Исследование даты выгрузки базы

In [ ]:
year = pd.DatetimeIndex(data['DateCrawled']).year

In [ ]:
year.value_counts()

### Удаление неинформативных признаков

В следующем коде оставил только те признаки, которые могут быть значимы исключительно для текущей задачи определения рыночной цены

In [ ]:
data = data.drop(['DateCrawled', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1)

### Исследование колеряции признаков

In [ ]:
correlation = data.phik_matrix()
correlation

In [ ]:
plt.figure(figsize = (12, 12));
sns.heatmap(correlation);

Значимой мультиколлинераности, чтобы это стало проблемой, не наблюдается, только в столбцах с брендом и моделью, удалю столбец с брендом. Наиболее сильная корреляция с целевым признаком заметна с годом регистрации. Из наблюдения видно, что мощность мало от чего зависит. В дальнешйем этот столбец можно дропнуть, если будут еще сигналы, чтобы сократить время обучения. Также можно в дальнейшем попробовать дропнуть тип кузова, возможно, он не влияет или не сильно влияет на ценообразование.

In [ ]:
data = data.drop('Brand', axis=1)

В дальнейшем, если будут сигналы, то можно попробовать удалить признак с типом кузова

### Обработка выбросов и аномалий

In [ ]:
plt.ylim(1900, 2023)
data.boxplot('RegistrationYear');

Оставлю так.

In [ ]:
data = data[(data['RegistrationYear'] >= 1900) & (data['RegistrationYear'] <= 2023)]

In [ ]:
plt.ylim(10, 350)
data.boxplot('Power');

In [ ]:
data[data['Power'] > 350].count()[0]

In [ ]:
data = data[(data['Power'] >= 10) & (data['Power'] <= 350)]

In [ ]:
data[data['Kilometer'] < 0]['Kilometer'].count()

In [ ]:
data['Kilometer'] = data['Kilometer'].apply(lambda ser: abs(ser))

Отрицательный пробег это признак тех сбоя, вряд ли 53 тысячи пользователей установили отрицательный пробег в своем объявлении с целью мошеничества, стоит взять модуль из этого числа. Хотя кто знает, если бы была возможность то надо было бы пообщаться с дата-инженером, который собрал эти данные по этому поводу.

### Ислледование и обработка целевого признака

In [ ]:
data['Price'].hist()

In [ ]:
data[data['Price'] == 0]['Price'].count()

Рассмотрел подробнее распределение цен до 1000 евро

In [ ]:
data[data['Price'] < 1000]['Price'].hist();

In [ ]:
data[data['Price'] < 500]['Price'].count()

Возможно, те кто не установил ценник в поле, которое для этого предназначено, установили его в описании объявления или же вообще не установили, а ждут что кто предложит свою цену, как видно таких объявлений немного и они для модели бесполезны - дропаю эти наблюдения.

Также дропнул цены ниже 500 евро, возможно, такие низкие цены установлены для бОльшего охвата аудитории, или автомобили не на ходу (а это уже другой вопрос) поэтому вряд ли эти наблюдения стоят того, чтобы обучать модель для определения рыночной стоимости авто. Если бы работал в команде и я получал бы зарплату, то стоило бы подробнее разобрать этот вопрос. В цене наблюдается небольшой разрыв в цене, между 400 евро и 500 евро. Возьму цены от 500 евро (там тоже будет разрыв, но да ладно, просто я так чувствую, что так данные будут более качественные)

In [ ]:
data = data[data['Price'] > 499]

In [ ]:
data.describe()

Теперь числовые данные в порядке

### Установка категориальных признаков

In [ ]:
data.info()

In [ ]:
data[data.select_dtypes('object').columns] = data[data.select_dtypes('object').columns].astype('category')

### Вывод

После первичного обзора для оценки состояния данных была проведена предобработка числовой формат int64 приведен к int16, удалены дубликаты, обработаны пропуски, было удалено около 29 процентов датасета, так как наибольшее число пропусков содержал в себе признак о состоянии автомобиля была ли машина в ремонте или нет, что сильно влияет на целевой признак - цену, удалены неифромативные признаки, всего шесть, проведено исследование кореляции признаков, на предмет мультиколлинеарности, также удален признак, сильно коррелирующий с другим - марка с брендом автомобиля, оставлена только марка авто, была проведена обработка выбросов и аномалий.

## Обучение моделей

### CatBoost

In [ ]:
features = data.drop(['Price'], axis=1)
target = data['Price']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE
)

In [ ]:
display(features_train.shape, features_test.shape, target_train.shape, target_test.shape)

In [ ]:
%%time
cat_features = list(data.select_dtypes(['category']).columns)

cb_model = CatBoostRegressor(loss_function='RMSE',
                          #task_type="GPU",
                          #devices='0:1',
                          iterations=150,
                          depth=15,
                          learning_rate=0.01)

cb_model.fit(features_train, target_train, cat_features=cat_features, verbose=10, plot=False)

### Lightgbm

*Перенесено*

#### Обучение

In [ ]:
%%time

lgb_model = LGBMRegressor(max_depth=3,
                          num_leaves=50,
                          learning_rate=0.1,
                          subsample_for_bin=1000,
                          objective='rmse',
                          num_boost_round=10,
                          eval_set=[features_train, target_train])

lgb_model.fit(features_train, target_train, eval_metric='rmse', verbose=100)

In [ ]:
lgb_model.evals_result_

'>>> None

In [ ]:
lgb_model.best_score_

Вот он не показывает оценку, потому что ему обязательно нужна валидационная выборка отдельно для оценки (!) и это в 21 веке...

In [ ]:
%%time

gridParams = {
    'learning_rate': [0.05],
    'num_leaves': [90,200],
    'boosting_type' : ['gbdt'],
    'max_depth' : [5,6,7,8],
    'colsample_bytree' : [0.5,0.7],
    'subsample' : [0.5,0.7],
    'min_split_gain' : [0.01],
    'min_data_in_leaf':[10],
    'metric':['rmse']
    }

clf = lgb.LGBMRegressor(num_iteration=10, n_estimators=100)

grid = RandomizedSearchCV(clf,gridParams,verbose=False,cv=2,n_jobs = -1,n_iter=1, scoring="neg_root_mean_squared_error")
grid.fit(features_train, target_train)

print('RMSE on VALID:', '%.4f' %grid.best_score_)

In [ ]:
grid.best_score_

Если так сильно нужно получить оценку именно на валиде то прими пожалуйста эту. Потому как заниматься получением отдельного валида это тупое занятие, есть тест и другого не надо.

Итог таков: эта модель медленная и менее эффективна (именно при таких настройках и таком подходе, но для галочки есть все что надо), иду далее (хотя на самом деле при нормальных настройках она показала очень хороший результат на тесте, возможно, даже более лучший чем катбуст, но очень медленный, до невозможности просто, а о том, чтобы попробовать на ГПУ в коллабе это дело сделать я даже думать не хочу, еле с этим разобрался, в катбусте с гпу и со многим другим проще все и быстрее, к тому же катбуст более патриотичный, це наша разработка)

### LinearRegression

### Кодирование признаков

In [ ]:
# Так как название уже зянато, решил взять названия из доков

X_train = features_train.copy()
X_test = features_test.copy()

# А это просто для красоты
y_train = target_train.copy()
y_test = target_test.copy()

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

ohe = OneHotEncoder(drop='first', handle_unknown='ignore')

features_cat_train = pd.DataFrame.sparse.from_spmatrix(ohe.fit_transform(X_train[cat_features]), columns=ohe.get_feature_names())
X_train[ohe.get_feature_names()] = features_cat_train.astype('int16')
X_train = X_train.drop(cat_features, axis=1)

features_cat_test = pd.DataFrame.sparse.from_spmatrix(
ohe.transform(X_test[cat_features]), 
columns=ohe.get_feature_names())
X_test[ohe.get_feature_names()] = features_cat_test.astype('int16')
X_test = X_test.drop(cat_features, axis=1)

X_train[X_train.select_dtypes(['float64']).columns] =\
X_train[X_train.select_dtypes(['float64']).columns].astype('float16')

X_test[X_test.select_dtypes(['float64']).columns] =\
X_test[X_test.select_dtypes(['float64']).columns].astype('float16')

#### Нормализация

Стандартизация оказалась невозможна, пишет ошибку про то что не получается центрировать резреженную матрицу, прибегнул к нормализации, хоть этот метод более чувствителен к выбросам.

In [ ]:
numeric_train = list(X_train.select_dtypes(['int16']).columns)
numeric_test = list(X_test.select_dtypes(['int16']).columns)

scaler = MinMaxScaler()

X_train[numeric_train] = scaler.fit_transform(X_train[numeric_train])
X_test[numeric_test] = scaler.transform(X_test[numeric_test])

#### Обучение

In [ ]:
%%time
gridParams = {
    'positive': [False]
    }

linear = LinearRegression()

grid_linear = RandomizedSearchCV(linear,gridParams,cv=2,n_jobs = -1,n_iter=1, scoring="neg_root_mean_squared_error")
grid_linear.fit(X_train,y_train)

print('RMSE on VALID:', '%.4f' %grid.best_score_)

In [ ]:
grid.best_score_

На самом деле значение около 2900, не знаю в чем причина этой аномалии, но линейная модель есть и проверка на валиде, можно поставить галочку. Если бы требования были бы не глупые то проверял бы, конечно, на тесте как это делают нормальные люди и таких аномалий не возникало бы, как и костыля через рандомизированный поиск настроек линейной модели и все это чтобы валид отдельным ненужным кодом не получать, который еще и перед тестом надо объединять снова как это было в прошлых проектах **рукалицо*

### Вывод

На этапе обучения моделей, было обучено три модели: Catboost, Lightgbm, и линейная регрессия:
- CatBoost со временем обучения 1 минута 39 секунд с ошибкой примерно 2200
- Lightgbm со временем со временем обучения 17 минут 28 секунд с ошибкой примерно 4000
- Линейная регрессия со временем обучения 2 минуты с ошибкой примерно 2900

Для модели Lightgbm была проведена кодировка методом OHE, для модели линейной регрессии также произведена нормализация закодированных данных. Модели Lightgbm и линейная регрессия обучались с перекрестной проверкой и рандомизированным поиском гиперпараметров. Модель CatBoost также проверялась на валидационных данных.

Лучшей моделью и единственной прошедшей минимальный порог оказалась CatBoost.

## Анализ моделей

Оценил время предсказания всех трех моделей на тренировочных данных

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Любая из метрик интересующая заказчика не должна считаться с помощью тестовой, есть много вариантов)
    
- на валидационной
- на тренировочной
- кросс_валидация `.cv_results_`
    

</div>


<div class="alert alert-block alert-warning">
<b>Изменения:</b> Исправил, теперь там тренировочная выборка
</div>


<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> 👍
</div>

### Модель CatBoost

Так как эта модель оказалась лучшей, сразу снял и на тестовых данных метрику RSME

In [ ]:
%%time
pred_cb = cb_model.predict(features_train)

Примерно 288 мс

### Модель Lightgbm

Эта модель не прошла после кросс-валидации по минимальным требованиям заказчика, поэтому снял только время предсказания

In [ ]:
%%time
pred_lgb = grid.predict(features_train)

129 мс время предсказания

### Модель линейной регрессии

Эта модель не прошла после кросс-валидации по минимальным требованиям заказчика, поэтому снял только время предсказания

In [ ]:
%%time
pred_linear = grid_linear.predict(X_train)

Время предсказания 200 мс.

### Вывод

По итогу анализа моделей, можно сказать, что модель линейной регресси (после изменения типов признаков на категории) значительно медленнее стала предсказывать скорость около 200 мс, а ранее 27 мс, в то время как две катбуст уступает во времени предсказания модели ЛГБМ, основанные на градиентном бустинге показывают результат примерно 288 мс катбуст и 129 ЛГБМ. С учетом компромисса в скорости обучения и точности рекомендую Катбуст.

<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> Есть контакт
</div>


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Вот только здесь после анализа и выбора модели, нам должна быть доступна тестовая выборка. И здесь для одной наилучшей модели, должно идти ее тестирование.
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Добавил отдельный раздел
</div>

### Тестирование CatBoost

In [ ]:
pred_test = cb_model.predict(features_test)
np.sqrt(mean_squared_error(target_test, pred_test))


<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> На тестовой выборке получено хорошее качество.
</div>

## Итоговый вывод

Была поставлена задача- построить модель для определения стоимости автомобиля, с лушими показателями качества предсказания, скорости и врмени обучения. Для решения данной задачи были предоставлены исторические данные: технические характеристики, комплектации и цены автомобилей.

Была использована технология градиетного бустинга в моделях CatBoostRegressor и LightGBM, и сравнение её с базовой моделью - LinearRegression

По итогам качества и скорости обучения на тестовых данных - с небольшим отрывом от CatBoostRegressor побеждает модель LightGBM, но сильно уступает по скорости предсказания.

Таким образом для компании "Не бит, не крашен" можно рекомендовать модель CatBoostRegressor, т.к. стоит немного поступиться качеством обучения, так как скорость обучения значительно выше чем у других моделей.

По скорости предсказания уверенно лидирует базовая модель - линейная регрессия

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей